In [ ]:
%%capture
# 필요한 패키지 일괄 설치 (코랩 등 환경에서 실행 시)
!pip install -q crewai==0.36.0 pandas>=2.0.0 openai>=1.0.0 langchain

# Step 1: Data Preprocessing for AI Vet Consultant

이 노트북에서는 AI 수의사 상담 시스템을 위한 데이터를 준비합니다.
CSV 파일에 있는 반려동물의 증상과 정보를 AI 모델이 이해하기 쉬운 **프롬프트 형식**으로 변환하는 과정을 실습합니다.

In [1]:
import pandas as pd
import os

# 데이터 경로 설정 (더 큰 데이터셋 사용)
INPUT_CSV_PATH = "../00_datasets/dr.tail/Dr.tail_210.CSV"
OUTPUT_DIR = "./data"

# 출력 디렉토리 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Created directory: {OUTPUT_DIR}")

## 1. 데이터 로드 및 확인
Pandas를 사용하여 CSV 파일을 읽고 데이터 구조를 확인합니다. 실습을 위해 5개만 샘플링합니다.

In [2]:
try:
    df_raw = pd.read_csv(INPUT_CSV_PATH)
    df = df_raw.head(5).copy()
    print(f"Successfully loaded dataset. Using first {len(df)} cases for exercise.")
    display(df.head(2))
except FileNotFoundError:
    print(f"Error: File not found at {INPUT_CSV_PATH}")

Successfully loaded dataset. Using first 5 cases for exercise.


,Case_ID,symptom,pet_information,issue_detailes,more_details,answer,Species
0,2,Diarrhea,Name\nLucyfur\nSpecies\nDog\nBreed\nPortuguese...,Main Concern\nDiarrhea\nColor\nGreen\nFrequenc...,She has had bad stinky gas for days. Eating gr...,The symptoms you're describing in your Portugu...,dog
1,3,Diarrhea,Name\nBow\nSpecies\nDog\nBreed\nPyrenean Sheph...,Main Concern\nDiarrhea\nColor\nBrown\nFrequenc...,"Hey, Like spray shits diarrhea. Every time he ...",Diarrhea in a 3-month-old Pyrenean Shepherd pu...,dog


## 2. 프롬프트 포맷팅 함수 작성 (실습)
각 행(Row)의 정보를 하나의 완성된 질문(Prompt) 텍스트로 합치는 함수를 작성해 봅시다.

**목표 형식:**
```
증상 (Symptom):
[증상 내용]

반려동물 정보 (Pet Information):
[반려동물 정보]

상세 문제 (Issue Details):
[상세 문제]

보호자 추가 설명 (More Details from Guardian):
[보호자 추가 설명]
```

In [4]:
def format_input_from_row(row):
    """Pandas 데이터프레임의 한 행을 받아 AI 입력 형식 문자열로 변환합니다."""
    
    # 1. 데이터 추출 (없는 경우 빈 문자열 처리)
    symptom = str(row.get('symptom', ''))
    pet_info = str(row.get('pet_information', ''))
    issue_details = str(row.get('issue_details', ''))
    more_details = str(row.get('more_details', ''))
    
    # 2. 포맷팅 (여기에 코드를 작성하세요)
    full_description = f"""
증상 (Symptom):
{symptom}

반려동물 정보 (Pet Information):
{pet_info}

상세 문제 (Issue Details):
{issue_details}

보호자 추가 설명 (More Details from Guardian):
{more_details}
"""
    return full_description.strip()

# 테스트: 첫 번째 행으로 결과 확인
if not df.empty:
    sample_row = df.iloc[0]
    print(format_input_from_row(sample_row))

증상 (Symptom):
Diarrhea

반려동물 정보 (Pet Information):
Name
Lucyfur
Species
Dog
Breed
Portuguese Podengo Pequeno
Age
3 years
Gender
Female
Status
Intact

상세 문제 (Issue Details):


보호자 추가 설명 (More Details from Guardian):
She has had bad stinky gas for days. Eating grass and puked.


## 3. 전체 데이터 변환 및 저장
모든 데이터에 이 함수를 적용하고, 나중에 사용할 수 있도록 저장합니다.

In [5]:
# 새로운 컬럼 'formatted_prompt' 생성
df['formatted_prompt'] = df.apply(format_input_from_row, axis=1)

# 필요한 컬럼만 선택하여 저장 (Prompt와 정답 Answer)
df_processed = df[['formatted_prompt', 'answer']]

output_path = os.path.join(OUTPUT_DIR, "processed_sample.csv")
df_processed.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"Saved {len(df_processed)} processed cases to {output_path}")
display(df_processed.head(1))

Saved 5 processed cases to ./data\processed_sample.csv


,formatted_prompt,answer
0,증상 (Symptom):\nDiarrhea\n\n반려동물 정보 (Pet Information):\nName\n...,The symptoms you're describing in your Portugu...


# Step 2: Single Agent Consultation

이 단계에서는 CrewAI의 가장 기본적인 단위인 **Single Agent(단일 에이전트)**를 만들어 봅니다.
복잡한 도구 없이, LLM의 지식만으로 반려동물의 증상을 분석하고 조언을 제공하는 '종합 수의사' 에이전트를 구현합니다.

In [6]:
# 필요한 라이브러리 설치 (최초 1회 실행)
try:
    import crewai
except ImportError:
    print("Installing crewai and dependencies...")
    !pip install crewai pandas openai

import os
import pandas as pd
from crewai import Agent, Task, Crew, LLM

# 환경 변수 설정 (API 키 하드코딩)
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"\nos.environ["OPENAI_MODEL_NAME"] = "gpt-5-mini" # 기본 모델 설정

## 1. LLM 설정
에이전트가 사용할 두뇌(LLM)를 설정합니다. 여기서는 gpt-5-mini를 사용합니다.

In [7]:
# LLM 설정
my_llm = LLM(
    model="gpt-5-mini",
    temperature=1
)

## 2. Agent 정의 (실습)
**`Agent`**는 특정 역할을 수행하는 AI 직원입니다.
- **Role**: 직책 (예: Pet Health Consultant)
- **Goal**: 목표 (예: 증상을 분석하여 조언 제공)
- **Backstory**: 배경 설정 (예: 10년 경력의 수의사...)

In [8]:
pet_consultant = Agent(
    role="종합 반려동물 건강 컨설턴트",
    goal="""
        보호자가 제공한 반려동물의 증상을 분석하고 종합적인 수의학 상담을 제공합니다.
        긴급도를 평가하고, 예상되는 원인을 제시하며, 실질적인 조언을 한국어로 작성합니다.
    """,
    backstory="""
        당신은 따뜻한 마음을 가진 경험 풍부한 수의사입니다.
        걱정하는 반려인들을 위해 복잡한 의학적 문제를 알기 쉬운 말로 설명하는 데 탁월한 능력을 갖추고 있습니다.
        당신은 항상 동물의 안전과 웰빙을 최우선으로 생각합니다.
    """,
    llm=my_llm,
    verbose=True
)

## 3. Task 정의 (실습)
**`Task`**는 에이전트가 수행해야 할 구체적인 작업입니다.
- **Description**: 작업 내용 (구체적으로 작성)
- **Expected Output**: 결과물의 형식

In [9]:
consultation_task = Task(
    description="""
        다음 반려동물 건강 케이스를 분석하세요:
        <case>
        {case_input}
        </case>

        당신의 분석에는 반드시 다음 내용이 포함되어야 하며 모든 내용은 한국어로 작성해야 합니다:
        1. 긴급도 평가 (위급, 응급, 긴급, 비긴급)
        2. 예상되는 원인 (감별 진단)
        3. 보호자를 위한 향후 권장 조치 및 조언
    """,
    expected_output="""
        마크다운 형식으로 작성된 전문적인 수의학 상담 리포트.
        '긴급도', '예상 원인', '권장 조치' 섹션을 포함해야 하며, 모든 내용은 한국어로 작성되어야 합니다.
    """,
    agent=pet_consultant
)

## 4. Crew 실행 (반복)
Agent와 Task를 하나로 묶어 **`Crew`**를 만들고 실행합니다.
Step 1에서 만든 5개의 데이터를 순회하며 모두 실행해 봅니다.

In [10]:
# 1. 데이터 로드
df_processed = pd.read_csv("./data/processed_sample.csv")
print(f"Loaded {len(df_processed)} cases.")

# 2. Crew 생성
single_agent_crew = Crew(
    agents=[pet_consultant],
    tasks=[consultation_task],
    verbose=True
)

# 3. 반복 실행 및 저장
outputs = []
for i, row in df_processed.iterrows():
    print(f"\n--- Processing Case #{i+1} ---")
    case_input = row['formatted_prompt']
    
    result = single_agent_crew.kickoff(inputs={"case_input": case_input})
    outputs.append(result.raw)
    
    # 개별 결과 저장
    with open(f"./data/single_result_{i}.md", "w", encoding="utf-8") as f:
        f.write(result.raw)

print("\nAll cases processed.")

Loaded 5 cases.

--- Processing Case #1 ---


╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  7e6f0356-23a2-4e95-a4d1-d039394e585b                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:                                                                                                          │
│          다음 반려동물 건강 케이스를 분석하세요:                                                                │
│          <case>                                                                                                 │
│          Symptom:                                                                                               │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Lucyfur                                                                                                        │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Portuguese Podengo Pequeno                                                                                     │
│  Age                                                                                                            │
│  3 years                                                                                                        │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Intact                                                                                                         │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She has had bad stinky gas for days. Eating grass and puked.                                                   │
│          </case>                                                                                                │
│                                                                                                                 │
│          당신의 분석에는 반드시 다음 내용이 포함되어야 하며 모든 내용은 한국어로 작성해야 합니다:               │
│          1. 긴급도 평가 (위급, 응급, 긴급, 비긴급)                                                              │
│          2. 예상되는 원인 (감별 진단)                                                                           │
│          3. 보호자를 위한 향후 권장 조치 및 조언                                                                │
│                                                                                                                 │
│  ID: 4dea38b7-8801-49a2-80ab-2c0c

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 종합 반려동물 건강 컨설턴트                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│          다음 반려동물 건강 케이스를 분석하세요:                                                                │
│          <case>                                                                                                 │
│          Symptom:                                                                                               │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Lucyfur                                                                                                        │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Portuguese Podengo Pequeno                                                                                     │
│  Age                                                                                                            │
│  3 years                                                                                                        │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Intact                                                                                                         │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She has had bad stinky gas for days. Eating grass and puked.                                                   │
│          </case>                                                                                                │
│                                                                                                                 │
│          당신의 분석에는 반드시 다음 내용이 포함되어야 하며 모든 내용은 한국어로 작성해야 합니다:               │
│          1. 긴급도 평가 (위급, 응급, 긴급, 비긴급)                                                              │
│          2. 예상되는 원인 (감별 진단)                                                                           │
│          3. 보호자를 위한 향후 권장 조치 및 조언                                                                │
│                                              

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 종합 반려동물 건강 컨설턴트                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # 수의학 상담 리포트 — Lucyfur (3세, 암컷, Portuguese Podengo Pequeno)                                         │
│                                                                                                                 │
│  안녕하세요. 걱정이 많으시겠어요. 제공해주신 정보(며칠간 심한 냄새의 방귀, 잔디 섭취 후 구토, 설사)를 바탕으로  │
│  Lucyfur의 상태를 분석해 드립니다.                                                                              │
│                                                                                                                 │
│  ## 긴급도                                                                                                      │
│  응급(빠른 수의학적 평가 권장)                                                                                  │
│  - 현재 상태는 즉시 생명을 위협하는 급성 위기(예: 쇼크, 심한 탈수, 의식 소실)는 아닌 것으로 보이나, 며칠간      │
│  지속되는 악취가 나는 가스와 설사, 잦은 잔디 섭취 및 구토는 추가 악화(탈수, 세균성·기생충성 감염 확산, 위장관   │
│  손상 등)의 위험이 있어 빠른 평가가 필요합니다.                                                                 │
│  - 특히 다음 증상이 있으면 즉시 응급진료:                                                                       │
│    - 설사에 혈액이 섞이거나(검붉거나 선홍색)                                                                    │
│    - 지속적/반복적인 구토(음수 불가)                                                                            │
│    - 식욕 급감 또는 전혀 먹지 않음                                                                              │
│    - 기력 저하(무기력, 쓰러짐), 고열, 복통(배를 만졌을 때 통증/불편)                                            │
│    - 눈에 보이는 탈수(잇몸 건조, 피부 탄력 저하)                                                                │
│  - 위의 증상 중 하나라도 보이면 즉시 병원 방문하십시오. 경미하고 활동적이며 식욕이 있으면서 설사만 있는         │
│  경우에는 가정관리 후 24–48시간 내 호전 여부를 관찰 가능하지만, 호전 없으면 진료 필요.                          │
│                                                                                                                 │
│  ## 예상 원인 (감별 진단)                                                                                       │
│  (가장 흔한 원인부터 제시)                                                                                      │
│  1. 식이성 위장관염(식탐/부적절한 음식 섭취, 급작스런 사료 변경, 이물질 섭취)                                   │
│     - 잔디 섭취 후 구토한 점과 냄새나는 가스는 흔히 식이 부적응이나 위장 자극과 관련.                           │
│  2. 기생충 감염(회충, 편충, Giardia 등)                                                                         │
│     - 특히 설사와 악취나는 가스는 Giardia 같은 기생충과 관련될 수 있음.                                         │
│  3. 세균성 장염(Clostridium spp., Salmonella 등)                                                                │
│     - 심한 악취의 가스와 설사, 때로 구토 동반.                                                                  │
│  4. 음식 부작용 또는 음식불내증 / 급성 식이과민반응                                                             │
│     - 새로운 간식이나 사람 음식 섭취 여부 확인 필요.                                                            │
│  5. 소화불량 또는 흡수장애(예: 췌장기능저하증은 드물지만 지방성 설사와 악취)                                    │
│  6. 이물(부분적 폐색) 또는 위장관 손상                                                                          │
│     - 반복적인 구토나 복통이 있으면 의심.                                                                       │
│  7. 중독(식물, 쓰레기, 독성 물질) — 드물지만 심각.                                                              │
│  8. 염증성 장질환(만성적 경과이면 고려)                                                                         │
│                                                                                                                 │
│ 

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 종합 반려동물 건강 컨설턴트                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # 수의학 상담 리포트 — Chewy (8세, 수컷, 중성화 완료, 혼합견)                                                  │
│                                                                                                                 │
│  안녕하세요. 이사 후 환경 변화로 심리적 스트레스도 받았겠지만, “오늘 설사 발견 / 먹지 않음·무기력·활동저하 /    │
│  배가 처진(팽만 또는 처짐) 상태”라는 증상 조합은 단순 스트레스성 반응일 수도 있지만 내과적                      │
│  문제(탈수·복부질환·감염·대사질환 등)가 숨어 있을 가능성도 있어 신속한 평가가 필요합니다. 따뜻한 마음으로 도움  │
│  드리겠습니다.                                                                                                  │
│                                                                                                                 │
│  ## 긴급도                                                                                                      │
│  긴급 — 가능한 한 빠른 수의학적 평가 권장(24시간 내로 병원 방문)                                                │
│  - 현재까지 호흡곤란, 쓰러짐, 반복적 구토, 혈변 등 심한 응급 증상이 보고되지는 않았으나,                        │
│    - 식욕상실과 전반적 무기력, 배의 이상(팽만·부종·현저한 처짐)은 심각한 기저질환(췌장염, 복수, 장폐색,         │
│  간·심장 질환, 전신감염 등)의 신호일 수 있습니다.                                                               │
│    - 특히 8세 이상 성견은 만성 질환 또는 급성 악화가 올 수 있으므로 지체하지 말고 24시간 이내 수의사 진료를     │
│  권합니다.                                                                                                      │
│  - 즉시 응급실로 가야 하는 상황(다음 중 하나라도 해당하면 즉시 응급 방문):                                      │
│    - 호흡 곤란, 푸르거나 창백한 잇몸, 의식 저하·경련·실신                                                       │
│    - 복부가 급격히 커지거나 만졌을 때 극심한 통증(낑낑거림)                                                     │
│    - 반복적인 구토로 수분 섭취 불가, 혈액 섞인 구토 또는 설사                                                   │
│                                                                                                                 │
│  ## 예상 원인 (감별 진단)                                                                                       │
│  (증상과 연령·환경변화를 고려한 가능성들 — 빈도/심각도 순으로)                                                  │
│  1. 스트레스성 또는 식이성 위장염(급성 식사부적응, 쓰레기·새로운 음식 섭취)                                     │
│  2. 기생충성 설사(회충, 편충, 지아르디아 등) — 이사·환경 변경 후 노출 가능성 증가                               │
│  3. 세균성 장염 또는 장내 미생물 불균형                                                                         │
│  4. 췌장염(특히 지방성 음식 섭취 후) — 식욕부진·무기력·복부 불편 동반 가능                                      │
│  5. 복수(복강 내 액체저류) 또는 복강 종양(간·복막·비장 질환) — “배가 처진/팽만” 느낌이 있는 경우                │
│  6. 장폐색(이물, 장중첩) — 복통·구토·심한 무기력 동반 시 의심                                                   │
│  7. 간·신장 질환 또는 대사성 질환(예: 당뇨, 전해질 불균형)                                                      │
│  8. 심부전(흉수·복수 유발)이나 혈액응고 문제, 패혈증 등 전신성 질환                                             │
│  9. 행동적 우울(환경 변화에 따른) — 의학적 문제 배제 후 고려                                                    │
│                                                                                                                 │
│  정확한 진단을 위해서는 신체검진 및 검사(혈액검사, 소변검사, 대변검사, 복부 방사선/초음파 등)가 필요합니다.     │
│                                                                                                                 │
│  ## 권장 조치 (보호자를 위한 단계별 실무 지침)                                                                  │
│                                                                                                                 │
│  1) 지금 당장 하실 일 (집에서)                                                                                  │
│  

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│                                                                                                                 │
│          다음 반려동물 건강 케이스를 분석하세요:                                                                │
│          <case>                                                                                                 │
│          Symptom:                                                                                               │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Chewy                                                                                                          │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Mixed                                                                                                          │
│  Age                                                                                                            │
│  8 years                                                                                                        │
│  Gender                                                                                                         │
│  Male                                                                                                           │
│  Status                                                                                                         │
│  Neutered or spayed                                                                                             │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  I just noticed it today,but we have moved and this was hard on him because my son has an Aussie doodle and me  │
│  and chewy raised her and trained her and we lived with my son for the last 2 or 4 years He used to eat         │
│  Everytime I feed peeper ,just cause he is stingy.But ever since it's just me and him and we have always been   │
│  alright on our own.Something is different this time he seems so depressed and just not interested in anything  │
│  even eating.He has the saggy belly now and haven't been gone 2 or three weeks.Im really worried.??He's never   │
│  not been by my side please tell me I'm just being overly cautious.not sure if that's the word I'm looking      │
│  for.He was my emotional support animal and if that might help on the sh

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  7e6f0356-23a2-4e95-a4d1-d039394e585b                                                                           │
│  Final Output: # 수의학 상담 리포트 — Chewy (8세, 수컷, 중성화 완료, 혼합견)                                    │
│                                                                                                                 │
│  안녕하세요. 이사 후 환경 변화로 심리적 스트레스도 받았겠지만, “오늘 설사 발견 / 먹지 않음·무기력·활동저하 /    │
│  배가 처진(팽만 또는 처짐) 상태”라는 증상 조합은 단순 스트레스성 반응일 수도 있지만 내과적                      │
│  문제(탈수·복부질환·감염·대사질환 등)가 숨어 있을 가능성도 있어 신속한 평가가 필요합니다. 따뜻한 마음으로 도움  │
│  드리겠습니다.                                                                                                  │
│                                                                                                                 │
│  ## 긴급도                                                                                                      │
│  긴급 — 가능한 한 빠른 수의학적 평가 권장(24시간 내로 병원 방문)                                                │
│  - 현재까지 호흡곤란, 쓰러짐, 반복적 구토, 혈변 등 심한 응급 증상이 보고되지는 않았으나,                        │
│    - 식욕상실과 전반적 무기력, 배의 이상(팽만·부종·현저한 처짐)은 심각한 기저질환(췌장염, 복수, 장폐색,         │
│  간·심장 질환, 전신감염 등)의 신호일 수 있습니다.                                                               │
│    - 특히 8세 이상 성견은 만성 질환 또는 급성 악화가 올 수 있으므로 지체하지 말고 24시간 이내 수의사 진료를     │
│  권합니다.                                                                                                      │
│  - 즉시 응급실로 가야 하는 상황(다음 중 하나라도 해당하면 즉시 응급 방문):                                      │
│    - 호흡 곤란, 푸르거나 창백한 잇몸, 의식 저하·경련·실신                                                       │
│    - 복부가 급격히 커지거나 만졌을 때 극심한 통증(낑낑거림)                                                     │
│    - 반복적인 구토로 수분 섭취 불가, 혈액 섞인 구토 또는 설사                                                   │
│                                                                                                                 │
│  ## 예상 원인 (감별 진단)                                                                                       │
│  (증상과 연령·환경변화를 고려한 가능성들 — 빈도/심각도 순으로)                                                  │
│  1. 스트레스성 또는 식이성 위장염(급성 식사부적응, 쓰레기·새로운 음식 섭취)                                     │
│  2. 기생충성 설사(회충, 편충, 지아르디아 등) — 이사·환경 변경 후 노출 가능성 증가                               │
│  3. 세균성 장염 또는 장내 미생물 불균형                                                                         │
│  4. 췌장염(특히 지방성 음식 섭취 후) — 식욕부진·무기력·복부 불편 동반 가능                                      │
│  5. 복수(복강 내 액체저류) 또는 복강 종양(간·복막·비장 질환) — “배가 처진/팽만” 느낌이 있는 경우                │
│  6. 장폐색(이물, 장중첩) — 복통·구토·심한 무기력 동반 시 의심                                                   │
│  7. 간·신장 질환 또는 대사성 질환(예: 당뇨, 전해질 불균형)                                                      │
│  8. 심부전(흉수·복수 유발)이나 혈액응고 문제, 패혈증 등 전신성 질환                                             │
│  9. 행동적 우울(환경 변화에 따른) — 의학적 문제 배제 후 고려                                                    │
│                                                                                                                 │
│  정확한 진단을 위해서는 신체검진 및 검사(혈액검사, 소변검사, 대변검사, 복부 방사선/초음파 등)가 필요합니다.     │
│                                                                                                                 │
│  ## 권장 조치 (보호자를 위한 단계별 실무 지침)                                                


All cases processed.


╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Step 3: Multi-Agent Crew Consultation

이 단계에서는 **Multi-Agent System(멀티 에이전트 시스템)**을 구축합니다.
단 하나의 '슈퍼맨' 에이전트 대신, 각자의 전문 분야를 가진 여러 에이전트(Triage, Vet, Advisor)가 협력하여 더 정확하고 풍부한 상담 결과를 만들어내는 과정을 실습합니다.

In [2]:
# 필요한 라이브러리 설치 (최초 1회 실행)
try:
    import crewai
except ImportError:
    print("Installing crewai and dependencies...")
    !pip install crewai pandas openai

import os
import pandas as pd
from crewai import Agent, Task, Crew, Process, LLM

# 환경 변수 설정 (API 키 하드코딩)
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"\nos.environ["OPENAI_MODEL_NAME"] = "gpt-5-mini"

# LLM 설정
my_llm = LLM(model="gpt-5-mini", temperature=1)

## 1. 전문 에이전트 정의 (실습)
세 명의 전문가를 고용합니다.

1.  **Triage Specialist (초진 담당)**: 응급도를 판단하고 분류합니다.
2.  **Veterinary Consultant (진단 담당)**: 증상을 바탕으로 의학적 진단을 내립니다.
3.  **Client Advisor (보호자 상담 담당)**: 어려운 의학 용어를 쉽게 풀어서 보호자에게 전달합니다.

In [3]:
# 1. Triage Agent
triage_agent = Agent(
    role="응급 분류 간호사 (Triage Specialist)",
    goal="증상을 바탕으로 반려동물 상태의 긴급도를 가장 먼저 분류합니다.",
    backstory="""당신은 경험 많은 동물병원 응급실 간호사입니다. 생명이 위급한 징후를 빠르게 파악하고 
    케이스를 위급, 응급, 긴급, 또는 비긴급 카테고리로 신속하게 분류하는 능력을 갖추고 있습니다.""",
    llm=my_llm,
    verbose=True
)

# 2. Vet Consultant Agent
vet_agent = Agent(
    role="진단 수의사 (Veterinary Consultant)",
    goal="증상과 응급도를 파악하여 예상되는 질병과 의학적 설명을 제공합니다.",
    backstory="""당신은 깊은 의학적 지식을 갖춘 전문 수의사입니다. 
    증상을 분석하여 가능한 질병 목록(감별 진단)을 논리적으로 추론해 냅니다.""",
    llm=my_llm,
    verbose=True
)

# 3. Advisor Agent
advisor_agent = Agent(
    role="상담가 (Pet Health Advisor)",
    goal="앞선 분석 결과를 종합하여 보호자를 위해 이해하기 쉽고 따뜻한 조언 리포트를 한국어로 작성합니다.",
    backstory="""당신은 고객 커뮤니케이션 전문가입니다. 복잡한 의학 정보를 
    걱정하는 보호자가 명확하게 이해할 수 있도록 친절하게 설명하고 가정 내 대처법을 덧붙입니다.""",
    llm=my_llm,
    verbose=True
)

## 2. Task 연결 (Workflow 설계)
각 에이전트가 수행할 작업을 정의하고, **이전 작업의 결과(Context)**를 다음 작업이 받을 수 있도록 설정합니다.
CrewAI의 `sequential` 프로세스는 Task 리스트의 순서대로 작업을 실행합니다.

In [4]:
# Task 1: Triage Assessment
triage_task = Task(
    description="""다음 케이스의 긴급도를 분석하세요: {case_input}. 
    Level 1 (위급) 부터 Level 5 (비긴급) 사이로 분류하고 그 이유를 한국어로 짧게 요약하세요.""",
    expected_output="한국어로 작성된 긴급도 분류 수준 및 그 판단 근거.",
    agent=triage_agent
)

# Task 2: Medical Diagnosis (Triage 기반 진단)
diagnosis_task = Task(
    description="""응급 분류 결과와 증상을 바탕으로 예상되는 진단명들을 나열하세요. 
    필요하다면 어떤 검사가 요구되는지도 한국어로 제안해 주세요.""",
    expected_output="한국어로 작성된 예상 질병 목록과 권장되는 의학적 검사 방법.",
    agent=vet_agent,
    context=[triage_task] # Triage 결과 입력 받음
)

# Task 3: Final Advice (최종 조언 작성)
advice_task = Task(
    description="""응급 분류 결과와 수의사의 진단 결과를 꼼꼼히 종합하여 보호자를 위한 최종 답변을 작성하세요. 
    긴급도, 예상 원인, 가정 내 대처법이 반드시 포함되어야 하며, 다정하고 친절한 어조를 사용하여 모든 문장을 한국어로 작성하세요.""",
    expected_output="한국어로 작성된 친절하고 완벽한 수의학 상담 최종 리포트.",
    agent=advisor_agent,
    context=[triage_task, diagnosis_task] # 이전 두 에이전트의 결과를 모두 종합
)

## 3. Crew 실행 (반복)
5개의 케이스에 대해 Multi-Agent 시스템을 실행합니다.

In [ ]:
# 데이터 로드
df_processed = pd.read_csv("./data/processed_sample.csv")

# Crew 생성
multi_agent_crew = Crew(
    agents=[triage_agent, vet_agent, advisor_agent],
    tasks=[triage_task, diagnosis_task, advice_task],
    process=Process.sequential, # 순차 실행
    verbose=True
)

# 반복 실행
for i, row in df_processed.iterrows():
    print(f"\n--- Processing Case #{i+1} ---")
    case_input = row['formatted_prompt']
    
    result = multi_agent_crew.kickoff(inputs={"case_input": case_input})
    
    # 결과 저장
    with open(f"./data/multi_result_{i}.md", "w", encoding="utf-8") as f:
        f.write(result.raw)

print("\nAll cases processed.")


--- Processing Case #1 ---


╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  6f2c5457-3780-48a2-b293-6baeb660853f                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: 다음 케이스의 긴급도를 분석하세요: Symptom:                                                              │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Lucyfur                                                                                                        │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Portuguese Podengo Pequeno                                                                                     │
│  Age                                                                                                            │
│  3 years                                                                                                        │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Intact                                                                                                         │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She has had bad stinky gas for days. Eating grass and puked..                                                  │
│      Level 1 (위급) 부터 Level 5 (비긴급) 사이로 분류하고 그 이유를 한국어로 짧게 요약하세요.                   │
│  ID: 3702b015-8ae6-4243-90a8-61550285a594                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 응급 분류 간호사 (Triage Specialist)                                                                    │
│                                                                                                                 │
│  Task: 다음 케이스의 긴급도를 분석하세요: Symptom:                                                              │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Lucyfur                                                                                                        │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Portuguese Podengo Pequeno                                                                                     │
│  Age                                                                                                            │
│  3 years                                                                                                        │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Intact                                                                                                         │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She has had bad stinky gas for days. Eating grass and puked..                                                  │
│      Level 1 (위급) 부터 Level 5 (비긴급) 사이로 분류하고 그 이유를 한국어로 짧게 요약하세요.                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 응급 분류 간호사 (Triage Specialist)                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  레벨 3 (긴급)                                                                                                  │
│                                                                                                                 │
│  이유: 수일간 지속된 악취 나는 가스와 설사, 풀 섭취 후 구토가 있어 소화기 문제(감염, 기생충, 식중독, 이물       │
│  등)로 진행되었을 가능성이 있고 탈수·전해질 이상으로 악화될 수 있으므로 24시간 이내 수의사 진찰이 필요합니다.   │
│  혈변·반복구토·무기력·식욕부진·탈수 징후가 있으면 즉시 응급처치(레벨 1–2)가 필요합니다.                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 응급 분류 간호사 (Triage Specialist)                                                                    │
│                                                                                                                 │
│  Task: 다음 케이스의 긴급도를 분석하세요: Symptom:                                                              │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Ivy                                                                                                            │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Dachshund                                                                                                      │
│  Age                                                                                                            │
│  13 years                                                                                                       │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Neutered or spayed                                                                                             │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She only had bad orangish diarhea one day. The next day she would not eat anything and she just layed around   │
│  yesterday i noticed her bresthing was shallow still not eating and laying around. Today shes wheezing and      │
│  wont walk and she looks a little swollen.                                                                      │
│      Level 1 (위급) 부터 Level 5 (비긴급) 사이로 분류하고 그 이유를 한국어로 짧게 요약하세요.                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 응급 분류 간호사 (Triage Specialist)                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  레벨 1 (위급)                                                                                                  │
│                                                                                                                 │
│  이유: 고령(13세) 개에서 설사 후 식욕부진·무기력·보행불가, 얕은 호흡과 쌕쌕거림(호흡곤란 증상), 몸의            │
│  붓기(부종/안구/얼굴 등)가 동반되어 있어 패혈증, 급성 심부전/폐부종, 아나필락시스 등 생명 위협적 상태 가능성이  │
│  큽니다. 즉시 응급수의사 진찰과 산소치료·안정·혈액검사·수액 등 응급처치가 필요하므로 즉시 응급병원으로 가세요.  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  다음 케이스의 긴급도를 분석하세요: Symptom:                                                                    │
│  Diarrhea                                                                                                       │
│                                                                                                                 │
│  Pet Information:                                                                                               │
│  Name                                                                                                           │
│  Ivy                                                                                                            │
│  Species                                                                                                        │
│  Dog                                                                                                            │
│  Breed                                                                                                          │
│  Dachshund                                                                                                      │
│  Age                                                                                                            │
│  13 years                                                                                                       │
│  Gender                                                                                                         │
│  Female                                                                                                         │
│  Status                                                                                                         │
│  Neutered or spayed                                                                                             │
│                                                                                                                 │
│  Issue Details:                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  More Details from Guardian:                                                                                    │
│  She only had bad orangish diarhea one day. The next day she would not eat anything and she just layed around   │
│  yesterday i noticed her bresthing was shallow still not eating and laying around. Today shes wheezing and      │
│  wont walk and she looks a little swollen.                                                                      │
│      Level 1 (위급) 부터 Level 5 (비긴급) 사이로 분류하고 그 이유를 한국어로 짧게 요약하세요.                   │
│  Agent:                                                                                                         │
│  응급 분류 간호사 (Triage Specialist)                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: 응급 분류 결과와 증상을 바탕으로 예상되는 진단명들을 나열하세요.                                         │
│      필요하다면 어떤 검사가 요구되는지도 한국어로 제안해 주세요.                                                │
│  ID: 3ef1aaa9-5b3e-496c-b8df-b1f706269078                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 진단 수의사 (Veterinary Consultant)                                                                     │
│                                                                                                                 │
│  Task: 응급 분류 결과와 증상을 바탕으로 예상되는 진단명들을 나열하세요.                                         │
│      필요하다면 어떤 검사가 요구되는지도 한국어로 제안해 주세요.                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Step 4: Evaluation (AI-as-a-Judge)

마지막 단계에서는 우리가 만든 AI 수의사들의 실력을 평가합니다.
사람이 직접 채점하는 대신, 또 다른 AI(LLM)에게 채점 기준표를 주고 평가를 맡기는 **Model-based Evaluation** 방식을 실습합니다.

In [1]:
import os
import pandas as pd
import json
from openai import OpenAI

# OpenAI 클라이언트 설정 (API Key 하드코딩)
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"\n
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

## 1. 정답 및 결과 로드
5개 케이스 각각에 대한 정답과 에이전트들의 결과를 불러옵니다.

In [2]:
# 정답 데이터 로드
df_truth = pd.read_csv("./data/processed_sample.csv")
print(f"Loaded {len(df_truth)} ground truth cases.")

# 결과 로드 함수
def load_result(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    return ""

# 모든 결과를 리스트로 로드
single_results = []
multi_results = []

for i in range(len(df_truth)):
    single_results.append(load_result(f"./data/single_result_{i}.md"))
    multi_results.append(load_result(f"./data/multi_result_{i}.md"))

Loaded 5 ground truth cases.


## 2. 평가 함수 작성
채점관 AI에게 줄 '채점 기준표'를 작성합니다.
정확성(Accuracy), 구체성(Completeness), 공감능력(Empathy) 세 가지 항목으로 평가하도록 지시합니다.

In [3]:
def evaluate_answer(answer, truth, problem_desc, criteria):
    prompt = f"""
    당신은 수의학 전문 평가자입니다.
    실제 수의사의 답변(Ground Truth)과 문제 상황(Problem Description)을 바탕으로 학생(AI)의 답변을 평가하세요.
    
    문제 상황 (Problem Description):
    {problem_desc}
    
    실제 수의사 정답 (Ground Truth):
    {truth}
    
    학생 AI 답변:
    {answer}
    
    평가 기준:
    {criteria}
    
    평가 결과는 반드시 아래와 같은 정확한 구조의 JSON 객체로 제공해야 합니다:
    {{
      "Accuracy": {{"score": 0, "reasoning": "text"}},
      "Completeness": {{"score": 0, "reasoning": "text"}},
      "Empathy": {{"score": 0, "reasoning": "text"}}
    }}
    (0을 0-10 사이의 점수로, text를 한국어로 작성된 점수 부여 사유로 교체하세요).
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-5-mini",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"Evaluation failed: {e}")
        return None



## 3. 평가 실행 (반복)
모든 케이스에 대해 점수를 매기고 평균을 냅니다.

In [4]:
criteria = """
1. Accuracy (정확성): 의학적 조언이 올바르고 안전한가?
2. Completeness (완전성): 긴급도, 원인, 가정 내 대처법을 모두 다루었는가?
3. Empathy (공감능력): 걱정하는 보호자에게 적절하고 친절한 어조인가?
"""

all_single_scores = []
all_multi_scores = []
all_vet_scores = []

for i in range(len(df_truth)):
    print(f"Evaluating Case #{i+1}...")
    ground_truth = df_truth.iloc[i]['answer']
    problem_desc = df_truth.iloc[i]['formatted_prompt']
    
    s_score = evaluate_answer(single_results[i], ground_truth, problem_desc, criteria)
    m_score = evaluate_answer(multi_results[i], ground_truth, problem_desc, criteria)
    v_score = evaluate_answer(ground_truth, ground_truth, problem_desc, criteria)
    
    if s_score: all_single_scores.append(s_score)
    if m_score: all_multi_scores.append(m_score)
    if v_score: all_vet_scores.append(v_score)

# 결과를 DataFrame으로 정리하여 평균 계산
def get_avg_scores(scores_list):
    if not scores_list: return {}
    avg = {'Accuracy': 0, 'Completeness': 0, 'Empathy': 0}
    for s in scores_list:
        avg['Accuracy'] += s['Accuracy']['score']
        avg['Completeness'] += s['Completeness']['score']
        avg['Empathy'] += s['Empathy']['score']
    for k in avg:
        avg[k] /= len(scores_list)
    return avg

avg_single = get_avg_scores(all_single_scores)
avg_multi = get_avg_scores(all_multi_scores)
avg_vet = get_avg_scores(all_vet_scores)

print("\n--- Average Single Agent Scores ---")
print(avg_single)
print("\n--- Average Multi Agent Scores ---")
print(avg_multi)
print("\n--- Average Vet (Ground Truth) Scores ---")
print(avg_vet)


Evaluating Case #1...
Evaluating Case #2...
Evaluating Case #3...
Evaluating Case #4...
Evaluating Case #5...

--- Average Single Agent Scores ---
{'Accuracy': 9.6, 'Completeness': 9.6, 'Empathy': 9.6}

--- Average Multi Agent Scores ---
{'Accuracy': 5.8, 'Completeness': 5.6, 'Empathy': 6.0}

--- Average Vet (Ground Truth) Scores ---
{'Accuracy': 9.4, 'Completeness': 8.6, 'Empathy': 9.4}


In [ ]:
# 시각화
import matplotlib.pyplot as plt

if avg_single and avg_multi and avg_vet:
    df_scores = pd.DataFrame({
        'Metric': list(avg_single.keys()),
        'Single Agent': list(avg_single.values()),
        'Multi Agent': list(avg_multi.values()),
        'Vet (Ground Truth)': list(avg_vet.values())
    })

    df_scores.set_index('Metric').plot(kind='bar', ylim=(0, 10), rot=0)
    plt.title('Performance Comparison (Avg of 5 Cases)')
    plt.ylabel('Score (0-10)')
    plt.show()
